In [1]:
from sympy import *
import numpy as np
import scipy.constants as c
import matplotlib.pyplot as plt
from tabulate import tabulate

import pandas as pd
import urllib.request

def lc_read_csv(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
lc_iso = livechart + "fields=ground_states"

def iso_molar(iso): #accepts string of Z number followed by atomic symbol
    iso_info = lc_read_csv(lc_iso + f"&nuclides={iso}")
    mu_iso_molar = iso_info["atomic_mass"][0]
    iso_molar = mu_iso_molar/10**6
    return iso_molar #returns molar mass in amu

def u_mev(u):
    MeV = u * (9.31494028*100)
    return MeV

In [350]:
#1

#Constants
sigma_a_U235 = 592.6
sigma_a_U238 = 2.382

sigma_f_U235 = 505.9
nu_U235 = 2.437

sigma_a_C = 3.861/1000 * (np.pi)**(1/2)/2

a_U238 = 2.73
c_U238 = .486

zeta_C = .158
sigma_s_C = 4.8

#Number Densities
N1_U235 = 1.5 / 100
N1_U238 = 98.5 / 100

N1_C = S('N1_C')
ratio1 = N1_C / (N1_U235 + N1_U238)


#K terms
eta1 = nu_U235*sigma_f_U235*N1_U235 / (sigma_a_U235*N1_U235 + sigma_a_U238*N1_U238)
f1 = (sigma_a_U235*N1_U235 + sigma_a_U238*N1_U238) / (sigma_a_U235*N1_U235 + sigma_a_U238*N1_U238 + sigma_a_C*ratio1)
epsilon1 = 1
p1 = exp(-a_U238/zeta_C * (N1_U238/(ratio1*sigma_s_C))**(1-c_U238))

k1 = eta1 * f1 * epsilon1 * p1


#Solving
bound = 600

r1 = np.zeros(bound)
k1_array = np.zeros(bound)

for i in range(bound):
    r1[i] = i
    k1_array[i] = k1.subs(N1_C, i)
    
k1_array[0] = 0
max_k1 = np.max(k1_array)
index1 = np.where(k1_array == max_k1)[0][0]

print('Max k w/ ratio =', r1[index1])
print('Max k', max_k1)

Max k w/ ratio = 578.0
Max k 1.0459201210315665


In [206]:
#2

N2_U235 = 3 / 100
N2_U238 = 97 / 100

M_to_U = 828

k2 = 1.26897
epsilon2 = 1
p2 = .78642
eta2 = 1.84117
f2 = .87640

s2 = S('s2')

#From Table 10.4
L2 = 55.4
Bm_sqd = (k2 - 1) / L2**2
Bg_sqd = 3 * (np.pi/s2)**2

eq2a = Eq(Bm_sqd, Bg_sqd)
soln2a = solve(eq2a)

soln2a[1]

581.257197033639

In [289]:
#3

sigma_s_H2O = 44.8
sigma_s_D2O = .509
sigma_s_Be = 6.1

sigma_a_H2O = .664
sigma_a_D2O = .0001133
sigma_a_Be = .0092

zeta_H2O = .920
zeta_D2O = .509
zeta_Be = .207

def optimum_fuel_to_mod_ratio(sigmaa, sigmas, zeta):
    r3 = S('r3')
    
    eta = (nu_U235*sigma_f_U235) / (sigma_a_U235)
    f = (sigma_a_U235) / (sigma_a_U235 + sigmaa*r3)
    p = exp(-a_U238/zeta * (1/(r3*sigmas))**(1-c_U238))
    k = eta*f*p
    
    bound = 70000
    spacing = 1
    
    r = np.arange(bound)
    ks = np.zeros(bound)
    
    for i in range(bound):
        ks[i] = k.subs(r3, i*spacing)
        if(ks[i] < ks[i-1]):
            max_k3 = ks[i-1]
            index3 = np.where(ks == max_k3)[0][0]
            return(spacing*r[index3])

In [290]:
#3 answers, Table 10.5
print('H2O', optimum_fuel_to_mod_ratio(sigma_a_H2O, sigma_s_H2O, zeta_H2O))
print('D2O', optimum_fuel_to_mod_ratio(sigma_a_D2O, sigma_s_D2O, zeta_D2O))
print('Be', optimum_fuel_to_mod_ratio(sigma_a_Be, sigma_s_Be, zeta_Be))
print('C', optimum_fuel_to_mod_ratio(sigma_a_C, sigma_s_C, zeta_C))

H2O 33
D2O 67883
Be 2962
C 7365


In [331]:
#4a

r4 = 40 #cm
ratio4 = 800 #H2O/U235

eta4 = nu_U235*sigma_f_U235 / sigma_a_U235
f4 = sigma_a_U235 / (sigma_a_U235 + sigma_a_H2O*ratio4)
p4 = exp(-0/zeta_H2O *(1/(ratio4*sigma_s_H2O))**(1-c_U238))

k4a = eta4*f4*p4
print('k infinity -', k4a)

k infinity - 1.09706202171205


In [354]:
#4b

Lsqd = 8.12
Bsqd = (np.pi/r4)**2
tau = 27

P_th_NL4 = 1 / (1 + Lsqd*Bsqd)
P_f_NL4 = exp(-Bsqd * tau)

keff4 = k4a * P_th_NL4 * P_f_NL4

print('keff4', keff4)

keff4 0.884451095064143


In [10]:
#5

Beta5, keff5 = S('Beta5, keff5')

p05 = 100
pf5 = 10000

t5 = 6*60 #min* sec/min

T5 = t5 / log(pf5/p05) #seconds
deltak = Beta5 * 12.8 / T5

insertion = deltak / Beta5

print('Insertion in $', insertion)

Insertion in $ 0.163739384390688
